In [12]:
import pandas as pd
from pathlib import Path
import os
from dotenv import load_dotenv


In [13]:
# --- IGNORE ---
# Environment variables
class EnvVarNames:
    PROJECT_ROOT = "PROJECT_ROOT"
    RAW_DATA_DIR = "RAW_DATA_DIR"
    TEMP_DIR = "TEMP_DIR"

# check whether environment variables are already set. If so, notify user that they will be overwritten in this process
for var_name in [EnvVarNames.PROJECT_ROOT, EnvVarNames.RAW_DATA_DIR]:
    if os.getenv(var_name) is not None:
        response = print(f"Environment variable '{var_name}' is already set and will be overwritten by loading .env file. Proceeding...")

load_dotenv(override=True)

Environment variable 'PROJECT_ROOT' is already set and will be overwritten by loading .env file. Proceeding...
Environment variable 'RAW_DATA_DIR' is already set and will be overwritten by loading .env file. Proceeding...


True

In [14]:
# setup paths 
PROJECT_ROOT_ENV_VALUE = os.getenv(EnvVarNames.PROJECT_ROOT)
assert PROJECT_ROOT_ENV_VALUE is not None, "PROJECT_ROOT environment variable is not set."
PROJECT_ROOT = Path(PROJECT_ROOT_ENV_VALUE)
RAW_DATA_DIR_ENV_VALUE = os.getenv(EnvVarNames.RAW_DATA_DIR)
assert RAW_DATA_DIR_ENV_VALUE is not None, "RAW_DATA_DIR environment variable is not set."
RAW_DATA_DIR = Path(RAW_DATA_DIR_ENV_VALUE)
filename = "Heat Stress Masterfile May 2024.xlsx"
file_path = Path(RAW_DATA_DIR / filename)

temp_dir = os.getenv(EnvVarNames.TEMP_DIR)
assert temp_dir is not None, "TEMP_DIR environment variable is not set."
TEMP_DIR = Path(temp_dir)

In [15]:
# Functions
def print_grid_of_values(values:list, items_per_row=3, align_columns=True):
    """display a list as a grid in the console"""
    if len(values) == 0:
        print("<no values>")
        return
    if align_columns:
        max_length = max([len(str(v)) for v in values])
        values = [str(v).ljust(max_length) for v in values]
    
    n = len(values)
    i = 0
    while(i<n):
        j = i + items_per_row
        [print(x, end=" | ") for x in values[i:j]]
        print()
        i =j

def show_unique_values_for_columns(df:pd.DataFrame, columns:list, max_unique_values_to_display=10):
    for col in columns:
        unique_values = df[col].unique()
        n_unique_values = len(unique_values)
        print(f"Column: {col} | Unique values: {n_unique_values}")
        if n_unique_values <= max_unique_values_to_display:
            print_grid_of_values(unique_values, items_per_row=5)
        else:
            print("Unique values exceed max display limit.")
        print("-"*40)

def explore_columns_unique_with_n_unique_values_or_less(df:pd.DataFrame, n:int):
    columns_to_explore = [col for col in df.columns if df[col].nunique() <= n]
    print_grid_of_values(columns_to_explore, items_per_row=3)
    show_unique_values_for_columns(df, columns_to_explore)

## Initial quick look at EA Masterfile
- How many columns?
- How many rows?
- How many unique values for key columns?

In [16]:
df = pd.read_excel(str(file_path), sheet_name="RF Ewe.ram data")

print(f"Dataframe shape: {df.shape}")
print(f"{df.shape[1]} columns\n{df.shape[0]} rows")


Dataframe shape: (1641, 118)
118 columns
1641 rows


**Filtering rows**

In [17]:
# Drop the weird row with mostly blanks (sex == 'M' and EID == '940 110012305918')
df = df[~((df['sex'] == 'M') & (df['EID'] == '940 110012305918'))]

At this stage, we're not intererested in males

In [18]:
# if 'sex' is 'm' (case insensitive), then drop the row
df_filtered = df[~df['sex'].str.lower().eq('m')].copy()


**Dropping Columns**

In [19]:
# Drop columns that have only a single unique value (including null)
single_value_columns = [col for col in df_filtered.columns if df_filtered[col].nunique(dropna=False) <= 1]
df_filtered.drop(columns=single_value_columns, inplace=True)
print(f"Dropped {len(single_value_columns)} columns with a single unique value.")
print_grid_of_values(single_value_columns, items_per_row=3)

Dropped 25 columns with a single unique value.
State                        | Site                         | Farm                         | 
Breed                        | sex                          | Paddock for 2023 (rams only) | 
Sensor serial 3 2023         | sensor serial 3 off 2023     | BLOOD TAKEN 2023             | 
2024 treatment (males)       | Sensor serial 1 2024         | Sensor serial 2 2024         | 
Sensor serial 3 2024         | GPS # 2024                   | LWC_during_joining_2024      | 
Sensor serial 1 off 2024     | Sensor serial 2 off 2024     | sensor serial 3 off 2024     | 
CS_lambing_2024              | WT_lambing_2024              | paddock_lambing_2024         | 
CS_marking 2024              | WT_marking _2024             | wet_dry_marking_2024         | 
Comments                     | 


In [20]:
for col in df_filtered.columns:
    print(f"{col}: {df_filtered[col].nunique()} unique values\n")

EID: 1596 unique values

EID VID: 689 unique values

VID: 1094 unique values

new RE tag 2023: 22 unique values

treatment: 6 unique values

Drop: 6 unique values

Paddock: 4 unique values

Weight for allocation 2022: 206 unique values

Paddock for 2022 (rams only): 1 unique values

date start of joining 2022: 2 unique values

WT start of joining 2022: 87 unique values

CS start of joining 2022: 14 unique values

Collar VEID 2022: 748 unique values

Sensor serial 1 2022: 730 unique values

Sensor serial 2 2022: 725 unique values

Sensor serial 3 2022: 1 unique values

GPS # 2022: 743 unique values

Date temp logger insterted 2022: 6 unique values

Temp logger # 2022: 749 unique values

Date end of joining 2022: 3 unique values

WT end of joining 2022: 76 unique values

CS at end of joining 2022: 22 unique values

LWC_during_joining_2022: 33 unique values

Collar EID 2022: 748 unique values

Sensor serial 1 off 2022: 736 unique values

Sensor serial 2 off 2022: 729 unique values

sensor

In [21]:
# write to excel
output_file_path = TEMP_DIR / "ea_masterfile_filtered.xlsx"
df_filtered.to_excel(output_file_path, index=False)

In [22]:
# for a given col, find values that appear more than n times
def find_frequent_values_in_column(df:pd.DataFrame, column_name:str, n:int):
    value_counts = df[column_name].value_counts()
    frequent_values = value_counts[value_counts > n]
    return frequent_values

find_frequent_values_in_column(df_filtered, 'EID', 1)

EID
940 110009541187    2
Name: count, dtype: int64